In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

In [3]:
all_df = pd.read_csv('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/AllData.csv', encoding='gb18030')
all_df.head()

,No.,Drop out,Outpatient_id,Immune_id,Age,Gender,Duration(Month),Distance to Hospital,ration of Treatment cost to Family income (Year),Pretreatment_ BCA,...,36 months_couch,36 months_wheeze,36 months_Total score,36 months_Effectiveness（%）,0 weeks_Drug score,16 weeks_Drug score,12 months_Drug score,18 months_Drug score,24 months_Drug score,36 months_Drug score
0,1,No,5121396,9449,8.0,1,36,1,0,0.88,...,0,0,5,0.852941,4,0,1,0.0,0,0
1,2,Pregnant,7002977,9447,33.0,1,18,0,1,0.28,...,0,1,18,0.485714,4,2,3,3.0,3,3
2,3,uneffective,7302433,9446,8.0,1,6,1,2,0.24,...,4,0,29,0.147059,4,3,3,2.0,3,3
3,4,pandemic,7324354,9445,8.0,0,13,1,0,0.56,...,0,0,12,0.675676,4,2,0,0.0,2,2
4,5,Pregnant,4822656,9444,32.0,1,18,0,1,0.20,...,0,0,16,0.515152,3,1,2,3.0,2,3


In [4]:
# 删除语义重复的列
new_df = all_df.drop(columns=['16 weeks_Effectiveness(%）', '12 months_Effectiveness（%）', '18 months_Effectiveness（%）',
                               '24 months_Effectiveness（%）', '36 months_Effectiveness（%）'])
new_df.head()
# all_df.shape

,No.,Drop out,Outpatient_id,Immune_id,Age,Gender,Duration(Month),Distance to Hospital,ration of Treatment cost to Family income (Year),Pretreatment_ BCA,...,36 months_chest distress,36 months_couch,36 months_wheeze,36 months_Total score,0 weeks_Drug score,16 weeks_Drug score,12 months_Drug score,18 months_Drug score,24 months_Drug score,36 months_Drug score
0,1,No,5121396,9449,8.0,1,36,1,0,0.88,...,0,0,0,5,4,0,1,0.0,0,0
1,2,Pregnant,7002977,9447,33.0,1,18,0,1,0.28,...,0,0,1,18,4,2,3,3.0,3,3
2,3,uneffective,7302433,9446,8.0,1,6,1,2,0.24,...,0,4,0,29,4,3,3,2.0,3,3
3,4,pandemic,7324354,9445,8.0,0,13,1,0,0.56,...,0,0,0,12,4,2,0,0.0,2,2
4,5,Pregnant,4822656,9444,32.0,1,18,0,1,0.20,...,0,0,0,16,3,1,2,3.0,2,3


In [5]:
def get_act(subject):
    act = np.zeros(5)

    if subject['Duration(Month)'].values[0] >= 4:
        act[0] = 1
    if subject['Duration(Month)'].values[0] >= 12:
        act[1] = 1
    if subject['Duration(Month)'].values[0] >= 18:
        act[2] = 1
    if subject['Duration(Month)'].values[0] >= 24:
        act[3] = 1
    if subject['Duration(Month)'].values[0] == 36:
        act[4] = 1

    return act


In [6]:
subject = new_df[1:2]
# subject
act = get_act(subject)
print(act)

[1. 1. 1. 0. 0.]


In [7]:
# 6个时间点，每个病人有6次状态，但只有5次行动
states = []
info = []
# print(new_df.tail(1).index[0])
acts = np.zeros([new_df.tail(1).index[0], 5])

for i in range(new_df.tail(1).index[0]):
    subject = new_df[i:i+1]
    acts[i] = get_act(subject)
    info.append([subject['Age'], subject['Gender'], subject['Distance to Hospital'],
                 subject['ration of Treatment cost to Family income (Year)']])
    phase = ['0 weeks', '16 weeks', '12 months', '18 months', '24 months', '36 months']
    subject_state = []
    for time in phase:
        single_state = np.asarray(subject.loc[i, lambda df: [col for col in df.columns if time in col]])
        # single_state = np.reshape(single_state, [3, 4])
        # print(single_state)
        subject_state.append(single_state)
    states.append(np.asarray(subject_state))
states = np.asarray(states)
info = np.array(info)
# info = np.squeeze(info)
print(states.shape)
print(info.shape)
# data

(205, 6, 12)
(205, 4, 1)


In [9]:
print(acts.shape)
print(acts[:, 0])

(205, 5)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [7]:
print(states[0][0])
print(states[0][1])
print(states[0][2])
print(states[0][3])
print(states[0][4])
print(states[0][5])

[5 5 4 5 4 4 3 0 4 0 34 4]
[4 5 5 4 4 0 2 0 5 0 29 0]
[2 3 2 2 4 2 0 0 0 0 15 1]
[3 2 2 2 3 2 0 0 0 0 14 0.0]
[2 2 1 1 2 0 0 0 0 0 8 0]
[2 1 0 2 0 0 0 0 0 0 5 0]


In [8]:
split = 0.75
i_split = int(states.shape[0] * split)

train_states = states[:i_split]
test_states = states[i_split:]

train_x_states = train_states[:, 0:-1]
train_y_states = train_states[:, 1:]

test_x_states = test_states[:, 0:-1]
test_y_states = test_states[:, 1:]

acts = np.expand_dims(acts, axis=2)
train_acts = acts[:i_split]
test_acts = acts[i_split:]

train_info = info[:i_split]
test_info = info[i_split:]

print(train_x_states.dtype)
print(train_y_states.dtype)
print(test_x_states.dtype)
print(test_y_states.dtype)

print(train_acts.dtype)
print(test_acts.dtype)
print(train_x_states.shape)
print(train_y_states.shape)

print(test_x_states.shape)
print(test_y_states.shape)
print(train_acts.shape)
print(test_acts.shape)
print(train_info.shape)
print(test_info.shape)

object
object
object
object
float64
float64
(153, 5, 12)
(153, 5, 12)
(52, 5, 12)
(52, 5, 12)
(153, 5, 1)
(52, 5, 1)
(153, 4, 1)
(52, 4, 1)


In [9]:
train_y = np.concatenate((train_acts, train_y_states), axis=2)
test_y = np.concatenate((test_acts, test_y_states), axis=2)
print(train_y.shape)
print(test_y.shape)
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/train_y', train_y)
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/test_y', test_y)

(153, 5, 13)
(52, 5, 13)


In [10]:
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/train_info', train_info)
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/test_info', test_info)
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/train_acts', train_acts)
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/test_acts', test_acts)
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/train_x_states', train_x_states)
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/train_y_states', train_y_states)
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/test_x_states', test_x_states)
np.save('/Users/fanwenxin/Desktop/ARGMAX/Immunotherapy/MyData/test_y_states', test_y_states)

In [10]:
# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler.fit_transform(train_x_states)
y_train = scaler_y.fit_transform(train_y_states)
Acts_train = scaler_y.fit_transform(train_acts)

X_test = scaler.transform(test_x_states)
y_test = scaler.transform(test_y_states)
Acts_test = scaler.transform(test_acts)

ValueError: Found array with dim 3. StandardScaler expected <= 2.